In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

C:\Users\athiq.ahmed\AppData2\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))

In [3]:
X[1:1]

array([], shape=(0, 200, 200, 1), dtype=uint8)

In [4]:
X =X/255.0

In [5]:
dense_layers = [0]
layer_sizes = [128]
conv_layers = [2]

In [6]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)
            model = Sequential()

            model.add(Conv2D(layer_size,(3,3), input_shape = X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.2))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model.fit(X,y,batch_size=32,epochs = 10, validation_split=0.3, callbacks=[tensorboard])
            
model.save('32x1-CNN.model')

2-conv-128-nodes-0-dense-1537203498
Train on 207 samples, validate on 90 samples
Epoch 1/10
207/207 [==============================] - 70s 337ms/step - loss: 0.8793 - acc: 0.5507 - val_loss: 0.6938 - val_acc: 0.4444
Epoch 2/10
207/207 [==============================] - 75s 362ms/step - loss: 0.6809 - acc: 0.6425 - val_loss: 0.6045 - val_acc: 0.7111
Epoch 3/10
207/207 [==============================] - 76s 366ms/step - loss: 0.6012 - acc: 0.7005 - val_loss: 0.5627 - val_acc: 0.7111
Epoch 4/10
207/207 [==============================] - 76s 369ms/step - loss: 0.5418 - acc: 0.7488 - val_loss: 0.5588 - val_acc: 0.7111
Epoch 5/10
207/207 [==============================] - 71s 345ms/step - loss: 0.4770 - acc: 0.7536 - val_loss: 0.5230 - val_acc: 0.7000
Epoch 6/10
207/207 [==============================] - 72s 349ms/step - loss: 0.4579 - acc: 0.7681 - val_loss: 0.5525 - val_acc: 0.6889
Epoch 7/10
207/207 [==============================] - 72s 349ms/step - loss: 0.3855 - acc: 0.8744 - val_loss:

In [7]:
import cv2
import tensorflow

CATEGORIES = ["Dettol", "Non-Dettol"]    # will use this to convert prediction num to string value

def prepare(filepath):
    IMG_SIZE = 200
    img_array = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)   # read in the image, convert to grayscale
    new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))   # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)     # return the image with shaping that TF wants.

In [8]:
model= tf.keras.models.load_model('32x1-CNN.model')

In [9]:
prediction = model.predict([prepare('dettol_pic 5.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dettol


In [10]:
prediction = model.predict([prepare('dettol_mix 9.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[1.9932495e-37]]
Dettol


In [11]:
prediction = model.predict([prepare('non dettol.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[1.]]
Non-Dettol


In [12]:
prediction = model.predict([prepare('non dettol 7.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dettol
